In [1]:
import numpy as np
import pickle
import os
from tqdm.notebook import tqdm

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.swnu_network_functions import (
    swnu_network_hyperparameter_search,
)

In [4]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [5]:
%run ../load_anno_mi.py

In [6]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime,speaker
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-11-02 00:00:13,-1
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-11-02 00:00:24,1
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-11-02 00:00:25,-1
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-11-02 00:00:34,1
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-11-02 00:00:34,-1


In [7]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)

sbert_embeddings.shape

(9699, 384)

# SWNU Network

## Obtaining path by looking at post history

We can obtain a path by looking at the history of each post. Here we look at the last 10 posts (and pad with vectors of zeros if there are less than 10 posts) including the current post.

We only want to consider paths that correspond to a client's utterance as we want to model a change in mood at that time. Their history will still contain the therapist's utterances too.

In [8]:
features = ["time_encoding", "timeline_index"]
standardise_method = ["z_score", None]
include_features_in_path = True
include_features_in_input = True

In [9]:
num_epochs = 100
dimensions = [15]
swnu_hidden_dim_sizes_and_sig_depths = [([12], 3), ([10], 4)]
ffn_hidden_dim_sizes = [[256, 256], [512, 512]]
dropout_rates = [0.1, 0.2]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

In [10]:
kwargs = {
    "num_epochs": num_epochs,
    "df": anno_mi,
    "id_column": "transcript_id",
    "label_column": "client_talk_type",
    "embeddings": sbert_embeddings,
    "y_data": y_data_client,
    "output_dim": output_dim_client,
    "dimensions": dimensions,
    "log_signature": True,
    "pooling": "signature",
    "swnu_hidden_dim_sizes_and_sig_depths": swnu_hidden_dim_sizes_and_sig_depths,
    "ffn_hidden_dim_sizes": ffn_hidden_dim_sizes,
    "dropout_rates": dropout_rates,
    "learning_rates": learning_rates,
    "BiLSTM": True,
    "seeds": seeds,
    "loss": loss,
    "gamma": gamma,
    "device": device,
    "features": features,
    "standardise_method": standardise_method,
    "include_features_in_path": include_features_in_path,
    "include_features_in_input": include_features_in_input,
    "path_indices": client_index,
    "split_ids": client_transcript_id,
    "k_fold": True,
    "patience": patience,
    "validation_metric": validation_metric,
    "verbose": False,
}

# w=5

In [11]:
size = 5

## GRP

In [12]:
(
    swnu_network_grp_kfold_5,
    best_swnu_network_grp_kfold_5,
    _,
    __,
) = swnu_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swnu_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swnu_network_grp_focal_2_5_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swnu_network_grp_focal_2_5_kfold_best_model.csv


In [13]:
swnu_network_grp_kfold_5.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_2823880/543207788.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_grp_kfold_5.groupby(


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.642593   
                                                       0.0005         0.616475   
                                                       0.0010         0.637739   
                                          0.2          0.0001         0.639336   
                                                       0.0005         0.630715   
                                                       0.0010         0.619796   
                           (512, 512)     0.1          0.0001         0.638250   
                                                       0.0005         0.620434   
                                                       0.0010         0.619221   
                                          0.2          0.0001         0.644381   
                                                       0.0005         0.619157   
                                                       0.0010         0.631418   
           (12,)           (256, 256)     0.1          0.0001         0.637612   
                                                       0.0005         0.635696   
                                                       0.0010         0.618455   
                                          0.2          0.0001         0.632248   
                                                       0.0005         0.630779   
                                                       0.0010         0.621711   
                           (512, 512)     0.1          0.0001         0.638697   
                                                       0.0005         0.626692   
                                                       0.0010         0.631992   
                                          0.2          0.0001         0.645785   
                                                       0.0005         0.630843   
                                                       0.0010         0.627586   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.517939   
                                                       0.0005         0.505092   
                                                       0.0010         0.508486   
                                          0.2          0.0001         0.519453   
                                                       0.0005         0.517751   
                                                       0.0010         0.498481   
                           (512, 512)     0.1          0.0001         0.517278   
                                                       0.0005         0.503690   
                                                       0.0010         0.495556   
                                          0.2          0.0001         0.515087   
                                                       0.0005         0.508058   
                                                       0.0010         0.503174   
           (12,)           (256, 256)     0.1          0.0001         0.514147   
                                                       0.0005         0.508992   
                                                       0.0010         0.510055   
                                          0.2          0.0001         0.510671   
                                                       0.0005         0.508133   
                                                       0.0010         0.507148   
                           (512, 512)     0.1          0.0001         0.516540   
                                                       0.0005         0.499256   
                                                       0.0010         0.511843   
                                          0.2      

In [14]:
swnu_network_grp_kfold_5.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_2823880/543207788.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_grp_kfold_5.groupby(


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.642593   
                                                       0.0005         0.616475   
                                                       0.0010         0.637739   
                                          0.2          0.0001         0.639336   
                                                       0.0005         0.630715   
                                                       0.0010         0.619796   
                           (512, 512)     0.1          0.0001         0.638250   
                                                       0.0005         0.620434   
                                                       0.0010         0.619221   
                                          0.2          0.0001         0.644381   
                                                       0.0005         0.619157   
                                                       0.0010         0.631418   
           (12,)           (256, 256)     0.1          0.0001         0.637612   
                                                       0.0005         0.635696   
                                                       0.0010         0.618455   
                                          0.2          0.0001         0.632248   
                                                       0.0005         0.630779   
                                                       0.0010         0.621711   
                           (512, 512)     0.1          0.0001         0.638697   
                                                       0.0005         0.626692   
                                                       0.0010         0.631992   
                                          0.2          0.0001         0.645785   
                                                       0.0005         0.630843   
                                                       0.0010         0.627586   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.517939   
                                                       0.0005         0.505092   
                                                       0.0010         0.508486   
                                          0.2          0.0001         0.519453   
                                                       0.0005         0.517751   
                                                       0.0010         0.498481   
                           (512, 512)     0.1          0.0001         0.517278   
                                                       0.0005         0.503690   
                                                       0.0010         0.495556   
                                          0.2          0.0001         0.515087   
                                                       0.0005         0.508058   
                                                       0.0010         0.503174   
           (12,)           (256, 256)     0.1          0.0001         0.514147   
                                                       0.0005         0.508992   
                                                       0.0010         0.510055   
                                          0.2          0.0001         0.510671   
                                                       0.0005         0.508133   
                                                       0.0010         0.507148   
                           (512, 512)     0.1          0.0001         0.516540   
                                                       0.0005         0.499256   
                                                       0.0010         0.511843   
                                          0.2      

In [15]:
best_swnu_network_grp_kfold_5

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.631609,0.517952,"[0.7513562386980108, 0.45882803523554194, 0.34...",0.510479,"[0.7844556324732537, 0.4281629735525375, 0.318...",0.529296,"[0.7209369577790631, 0.49422442244224424, 0.37...",None,0.673250,...,1,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.637165,0.510735,"[0.7649904706054831, 0.4227234753550543, 0.344...",0.505492,"[0.7757954207552781, 0.428087986463621, 0.3125...",0.518537,"[0.7544823597455177, 0.4174917491749175, 0.383...",None,0.669075,...,12,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.645977,0.523149,"[0.7655601659751037, 0.46760343481654953, 0.33...",0.518799,"[0.7851063829787234, 0.4437037037037037, 0.327...",0.528881,"[0.7469635627530364, 0.49422442244224424, 0.34...",None,0.672447,...,123,True,focal,2,True,5,Conv1d,None,concatenation,64


in path (with speaker): 0.5143628505176907

in path (no speaker): 0.522911065406166

in input (no speaker): 0.5125295634373418

both (no speaker): 0.5151677550731781

(w/ time_encoding) in path (with speaker): 0.5186878521492474

(w/ time_encoding) in path (no speaker): 0.5191477428145815

(w/ time_encoding) in input (no speaker): 0.5126337587709865

(w/ time_encoding) both (no speaker):  0.5197361095991764

In [16]:
best_swnu_network_grp_kfold_5["f1"].mean()

0.5172784687824037

In [17]:
best_swnu_network_grp_kfold_5["precision"].mean()

0.5115898822057466

In [18]:
best_swnu_network_grp_kfold_5["recall"].mean()

0.5255712951283561

In [19]:
np.stack(best_swnu_network_grp_kfold_5["f1_scores"]).mean(axis=0)

array([0.76063563, 0.44971832, 0.34148147])

In [20]:
np.stack(best_swnu_network_grp_kfold_5["precision_scores"]).mean(axis=0)

array([0.78178581, 0.43331822, 0.31966561])

In [21]:
np.stack(best_swnu_network_grp_kfold_5["recall_scores"]).mean(axis=0)

array([0.74079429, 0.46864686, 0.36727273])

# w=11

In [22]:
size = 11

## GRP

In [ ]:
(
    swnu_network_grp_kfold_11,
    best_swnu_network_grp_kfold_11,
    _,
    __,
) = swnu_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swnu_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
swnu_network_grp_kfold_11.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

In [ ]:
swnu_network_grp_kfold_11.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

In [ ]:
best_swnu_network_grp_kfold_11

In [ ]:
best_swnu_network_grp_kfold_11["f1"].mean()

In [ ]:
best_swnu_network_grp_kfold_11["precision"].mean()

In [ ]:
best_swnu_network_grp_kfold_11["recall"].mean()

In [ ]:
np.stack(best_swnu_network_grp_kfold_11["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_grp_kfold_11["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_grp_kfold_11["recall_scores"]).mean(axis=0)

# w=20

In [ ]:
size = 20

## GRP

In [ ]:
(
    swnu_network_grp_kfold_20,
    best_swnu_network_grp_kfold_20,
    _,
    __,
) = swnu_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swnu_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

In [ ]:
swnu_network_grp_kfold_20.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

In [ ]:
swnu_network_grp_kfold_20.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

In [ ]:
best_swnu_network_grp_kfold_20

In [ ]:
best_swnu_network_grp_kfold_20["f1"].mean()

In [ ]:
best_swnu_network_grp_kfold_20["precision"].mean()

In [ ]:
best_swnu_network_grp_kfold_20["recall"].mean()

In [ ]:
np.stack(best_swnu_network_grp_kfold_20["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_grp_kfold_20["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_grp_kfold_20["recall_scores"]).mean(axis=0)

# w=35

In [ ]:
size = 35

## GRP

In [ ]:
(
    swnu_network_grp_kfold_35,
    best_swnu_network_grp_kfold_35,
    _,
    __,
) = swnu_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swnu_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

In [ ]:
swnu_network_grp_kfold_35.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

In [ ]:
swnu_network_grp_kfold_35.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

In [ ]:
best_swnu_network_grp_kfold_35

In [ ]:
best_swnu_network_grp_kfold_35["f1"].mean()

In [ ]:
best_swnu_network_grp_kfold_35["precision"].mean()

In [ ]:
best_swnu_network_grp_kfold_35["recall"].mean()

In [ ]:
np.stack(best_swnu_network_grp_kfold_35["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_grp_kfold_35["precision_scores"]).mean(axis=0)

# w=80

In [ ]:
size = 80

## GRP

In [ ]:
(
    swnu_network_grp_kfold_80,
    best_swnu_network_grp_kfold_80,
    _,
    __,
) = swnu_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swnu_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

In [ ]:
swnu_network_grp_kfold_80.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

In [ ]:
swnu_network_grp_kfold_80.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

In [ ]:
best_swnu_network_grp_kfold_80

In [ ]:
best_swnu_network_grp_kfold_80["f1"].mean()

In [ ]:
best_swnu_network_grp_kfold_80["precision"].mean()

In [ ]:
best_swnu_network_grp_kfold_80["recall"].mean()

In [ ]:
np.stack(best_swnu_network_grp_kfold_80["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_grp_kfold_80["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_grp_kfold_80["recall_scores"]).mean(axis=0)

# w=110

In [ ]:
size = 110

## GRP

In [ ]:
(
    swnu_network_grp_kfold_110,
    best_swnu_network_grp_kfold_110,
    _,
    __,
) = swnu_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swnu_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

In [ ]:
swnu_network_grp_kfold_110.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

In [ ]:
swnu_network_grp_kfold_110.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

In [ ]:
best_swnu_network_grp_kfold_110

In [ ]:
best_swnu_network_grp_kfold_110["f1"].mean()

In [ ]:
best_swnu_network_grp_kfold_110["precision"].mean()

In [ ]:
best_swnu_network_grp_kfold_110["recall"].mean()

In [ ]:
np.stack(best_swnu_network_grp_kfold_110["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_grp_kfold_110["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_grp_kfold_110["recall_scores"]).mean(axis=0)